# Append Data
Reads a csv into a dataframe, chunks the dataframe, publishes the first chunk, then appends the remaining chunks. This showcases the **[`edit_features`](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html#arcgis.features.FeatureLayer.edit_features)** function to append data when working with a [SpatialDataFrame](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html#arcgis.features.SpatialDataFrame)

In [ ]:
import pandas as pd
import json
from arcgis.gis import GIS
from arcgis.features import SpatialDataFrame
from arcgis.geometry import Point

In [ ]:
# log in to your GIS
gis = GIS(username="mpayson_startups")

In [ ]:
# chunk a dataframe
def chunk_df(df, n=1000):
    for i in range(0, len(df), n):
        yield df.iloc[i:i + n].copy()

# constructs a point from a row or dictionary
def get_point_for_row(row, coord_fields):
    p_dict = {}
    p_dict['x'] = row[coord_fields['x']]
    p_dict['y'] = row[coord_fields['y']]
    if 'z' in coord_fields and coord_fields['z']:
        p_dict['z'] = row[coord_fields['z']]
    return Point(p_dict)

In [ ]:
# path to data
csv_path = "../test_data/NYC Inspections Geocoded.csv"

# maps 'x', 'y', 'z' fields to the appropriate CSV fields
coord_fields = {
    "x": "x",
    "y": "y",
    #"z": "<Z COLUMN (optional)>"
}

In [ ]:
# read csv and construct spatial dataframe
df = pd.read_csv(csv_path)
df_geom = df.apply(lambda row: get_point_for_row(row, coord_fields), axis=1)
sdf = SpatialDataFrame(data=df, geometry=df_geom)
len(sdf)

In [ ]:
# iterate through chunks to create and append data
lyr = None
for c_df in chunk_df(sdf, 50):
    if not lyr:
        item = c_df.to_featurelayer("MyFeatureService")
        lyr = item.layers[0]
    else:
        # THIS IS THE APPEND DATA PART
        fs = c_df.to_featureset()
        success = lyr.edit_features(adds=fs)
item